In [68]:
# install api interface at https://github.com/cwendt94/ff-espn-api/

import requests

# get hardcoded cookies from browser


# helper function from ff-espn-api
def checkRequestStatus(status: int) -> None:
    if 500 <= status <= 503:
            raise Exception(status)
    if status == 401:
        raise Exception("Access Denied")

    elif status == 404:
        raise Exception("Invalid League")

    elif status != 200:
        raise Exception('Unknown %s Error' % status)


year = 2020
league_id = 83174673


# fetch league. need some data from cookies to fill in here
swid = '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}'
espn_s2 = 'AEAy3mkAv%2FsnK2YbgEVRFKpOw6%2ByQVeXH5BDBOlzgAiBg646ZtRuneInbY1z9QYrt1Ws8r6Cc8pS0g%2BVi7HmU0b%2F0lxgkP9csOvk9S01pusHVMJOZ9GfMZblZ01U9NjD46R2N%2BI21guim1Lga4TfVKqE6rCLMJvQszv5ZKdFgjngR98gxiClp9R3ZxzJ7cTLJufuir4JGnzKZ9RwOCL1%2BOaFy8qFEGwEVNGkuMY4Hj4zSR%2BL9a5fatMYEWNXvk4muhbgckHixSZiqvSnL6pyniu98SRFEpigMIkWgSJ%2FTfNvCg%3D%3D'
cookies = {
    'espn_s2': espn_s2,
    'SWID': swid
}
ENDPOINT = "https://fantasy.espn.com/apis/v3/games/FFL/seasons/" + str(year) + "/segments/0/leagues/" + str(league_id)
params = ''

# test request
r = requests.get(ENDPOINT, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)

{'gameId': 1, 'id': 83174673, 'members': [{'displayName': 'espn04014086', 'id': '{23752840-5024-4BC3-B528-4050243BC3EE}', 'isLeagueManager': False}, {'displayName': 'YungFly', 'id': '{2A52739B-A4D2-440F-B78D-C1282294B36E}', 'isLeagueManager': False}, {'displayName': 'Ball, call, install', 'id': '{5E1655AA-B7E8-4112-8208-4024E2B806B4}', 'isLeagueManager': False}, {'displayName': 'espn57414091', 'id': '{86F13096-2F1D-44BC-B130-962F1DA4BCEA}', 'isLeagueManager': False}, {'displayName': 'broncoz16', 'id': '{89BABC79-3E89-4484-B7E0-91FE30264A36}', 'isLeagueManager': False}, {'displayName': 'typhe12', 'id': '{8BCE107A-C396-4216-A165-204DCBB7ACED}', 'isLeagueManager': False}, {'displayName': 'ESPNFAN6778113520', 'id': '{9DAF6913-3970-416F-BECB-D0DE77788ADB}', 'isLeagueManager': False}, {'displayName': 'shottysnipesonthepit', 'id': '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}', 'isLeagueManager': False}, {'displayName': 'JackMichaels55', 'id': '{AED6F6FD-B3C1-49EC-9329-C5E971023ED7}', 'isLeagueMana

In [69]:
"""
# fetch players, get general data about players, don't really remember what's in here
# this will print all players with last name starting with "a"s
params = {
    'scoringPeriodId': 3,
    'view': 'players_wl',
}
endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/players'
r = requests.get(endpoint, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)
"""

'\n# fetch players, get general data about players, don\'t really remember what\'s in here\n# this will print all players with last name starting with "a"s\nparams = {\n    \'scoringPeriodId\': 3,\n    \'view\': \'players_wl\',\n}\nendpoint = \'https://fantasy.espn.com/apis/v3/games/ffl/seasons/\' + str(year) + \'/players\'\nr = requests.get(endpoint, params=params, cookies=cookies)\nstatus = r.status_code\ncheckRequestStatus(status)\ndata = r.json()\nprint(data)\n'

In [70]:
import numpy as np

# fetch players, get scoring data
def get_player_data(week):
    params = {
        'scoringPeriodId': week
    }
    endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/segments/0/leagues/' + str(league_id) + '?view=mMatchup&view=mMatchupScore'
    r = requests.get(endpoint, params=params, cookies=cookies)
    status = r.status_code
    checkRequestStatus(status)
    d = r.json()
    return d

# get player scoring data for all weeks
import pandas as pd

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

# holder variable
data = []

# iterate teams
for week in range(1, 14):

    # get player data for that scoring period
    d = get_player_data(week)

    for tm in d['teams']:
        tmid = tm['id']

        # iterate players on teams
        for p in tm['roster']['entries']:
            name = p['playerPoolEntry']['player']['fullName']
            slot = p['lineupSlotId']
            pos  = slotcodes[slot]

            # injured status (need try/exc bc of D/ST)
            inj = 'NA'
            try:
                inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # grab projected/actual points
            proj, act = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != week:
                    continue
                if stat['statSourceId'] == 0:
                    act = stat['appliedTotal']
                elif stat['statSourceId'] == 1:
                    proj = stat['appliedTotal']

            data.append([
                week, tmid, name, slot, pos, inj, proj, act
            ])
    print('Week {} complete.'.format(week))

data = pd.DataFrame(data, columns=['Week', 'Team', 'Player', 'Slot', 'Pos', 'Status', 'Proj', 'Actual'])
#


Week 1 complete.
Week 2 complete.
Week 3 complete.
Week 4 complete.
Week 5 complete.
Week 6 complete.
Week 7 complete.
Week 8 complete.
Week 9 complete.
Week 10 complete.
Week 11 complete.
Week 12 complete.
Week 13 complete.


In [72]:
# load data from csv
data.to_csv('20201209_data.csv')
#data = pd.read_csv('20201209_data.csv')

In [141]:
# convenience functions
def get_player_performance_rank(player, week, only_starters=True):
    """ function that evaluates the positional ranking of a player among players owned (or just starters) that week
    I think this metric has better insight into how good a player is. random waiver players or bench players who
    pop off are better thought of as noise, assuming we are rational managers """

    # load data
    data = pd.read_csv('20201209_data.csv')
    if only_starters:
        data = data[data['Pos'] != 'Bench']
    
    # just work with data for that week
    week_data = data[data['Week'] == week]

    # get position of that player
    pos = week_data[week_data['Player'] == player]['Pos']

    # return nan if player didn't start that week
    # and the person had "only_starters" selected
    if pos.empty:
        print("Warning: player {} not in dataset for week {}. Returning NaN.".format(player, week))
        return np.NaN   

    # get scores of players of that position
    pos_data = week_data[week_data['Pos'] == pos.unique()[0]]

    # get scores and sort descending
    scores = pos_data['Actual'].unique()
    scores.sort()
    scores = scores[::-1]

    # get where player score is in rank of scores
    player_score = pos_data[pos_data['Player']==player]['Actual'].tolist()[0]
    rank = np.where(scores == player_score)[0][0]

    # increment cause zero indexing
    rank += 1

    return rank


def get_player_score(player, week, projected=False):
    """ Convenience function to get score of a player given data and week
    """
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]

    # get data for that player
    player_data = data[data['Player'] == player]

    # return nan if player didn't score in that dataset for whatever reason
    if player_data.empty:
        print("Warning: no player score for {} recorded for week {}.".format(player, week))
        return np.NaN

    # return score
    if projected:
        target = 'Proj'
    else:
        target = 'Actual'
    player_score = player_data[target].tolist()[0]

    # if player was on bye, score will be zero but there's no other indication
    if np.isnan(player_score):
        #print('Warning: getting score from player {} on bye week {}, returning 0'.format(player, week))
        player_score = 0

    return player_score


def was_player_started(player, week):

    # just work with data for that week
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return False

    # if player was in dataset as bench, he was on bench
    if player_data['Pos'] == 'Bench':
        return False

    # if player was on IR, he's basically on the bench
    if player_data['Pos'] == 'IR':
        return False

    # if not the above, then ya
    return True


def was_player_on_bench(player, week):
    # just work with data for that week
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return False

    # if player was in dataset as bench, he was on bench
    if player_data['Pos'] == 'Bench':
        return True

    # if player was on IR, he's basically on the bench
    if player_data['Pos'] == 'IR':
        return True
    
    # otherwise player started
    return False


def was_player_on_waivers(player, week):
    # just work with data for that week
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return True
    else:
        return False


def get_team_id_from_nickname(nickname):
    """ get the team id given a nickname for convenience """

    nicknames = {
        'Tyler': 1,
        'Ray': 4,
        'Blount': 8,
        'Jack': 7,
        'Poogz': 10,
        'Brian': 3,
        'Mitch': 2,
        'D\'vonne': 5,
        'Sam': 6,
        'Hogz': 9
    }

    if nickname not in nicknames.keys():
        print('Warning: nickname {} not recognized in list {}'.format(nickname, nicknames))
        return -1
    
    return nicknames[nickname]


def get_team_nicknames():

    nicknames = [
        'Tyler',
        'Ray',
        'Blount',
        'Jack',
        'Poogz',
        'Brian',
        'Mitch',
        'D\'vonne',
        'Sam',
        'Hogz'
    ]
    return nicknames


def get_team_starters(team, week, pos=None):
    """ get players a team started that week. team can be nickname or id. optional argument to get players of a certain position"""
    
    if type(team) == str:
        team_ndx = get_team_id_from_nickname(team)
    else:
        team_ndx = team

    # load data and just get data for that week/team
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    data = data[data['Team'] == team_ndx]

    # get players not on bench or IR
    data = data[data['Pos'] != 'Bench']
    starters = data[data['Pos'] != 'IR']

    if pos is not None:
        starters = starters[starters['Pos'] == pos]
        if starters.empty:
            print('Warning: position {} not found in starters for team {} week {}.'.format(pos, team, week))
            return []
    
    return starters['Player'].tolist()


def get_player_position(player):
    """ return position of player. heavy assumption player pos doesn't change """
    data = pd.read_csv('20201209_data.csv')
    pos = data[data['Player'] == player]['Pos'].tolist()[0]
    return pos


def get_team_color(team):
    if type(team) == str:
        team_ndx = get_team_id_from_nickname(team)
    else:
        team_ndx = team

    team_colors = np.array(plt.get_cmap('tab10').colors)
    return team_colors[team_ndx-1,:]



In [86]:
# analysis
import pandas as pd
import numpy as np

week = 1
team = 'Blount'
players = get_team_starters(team, week)
#get_team_id_from_nickname(team)

player = players[0]
get_player_score(player, week)
get_player_score(player, week, projected=True)
#get_player_performance_rank(player, week)
#get_player_position(player)

19.186692195

In [88]:
# iterate teams
teams = get_team_nicknames()

# get difference between perfornance and projected performance
teamsum = {}
luckiness = {}
for team in teams:
    
    teamsum[team] = []
    luckiness[team] = []

    for week in range(1, 14):

        # list of values for each week
        teamsum[team].append(0)
        luckiness[team].append(0)

        # iterate starters
        starters = get_team_starters(team, week)
        for starter in starters:
            score = get_player_score(starter, week)
            delta = score - get_player_score(starter, week, projected=True)

            # add delta to sum for that week
            teamsum[team][int(week) - 1] += score
            luckiness[team][int(week) - 1] += delta



In [143]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib qt

# some overall decoration settings
sns.set_style('darkgrid')
xvals = list(range(1,14))
linewidth=3


In [149]:
# plot team score by week
avg_list = []
for team in teamsum.keys():
    plt.plot(xvals, teamsum[team], label=team, color=get_team_color(team), linewidth=linewidth)
    avg_list.append(teamsum[team])

# plot average
avg_arr = np.array(avg_list)
plt.plot(xvals, avg_arr.mean(axis=0), color='k', linewidth=3, label='average')

# decorate
plt.legend(loc='lower center', ncol=5)
plt.ylabel('Team points')
plt.xlabel('Week')
plt.ylim([50, 225])
plt.xticks(xvals)
plt.title('Team score by week')

Text(0.5, 1.0, 'Team score by week')

In [154]:
# plot team score by week (actual - projected)
avg_list = []
for team in luckiness.keys():
    num_lucky = np.sum(np.array(luckiness[team]) > 0)
    plt.plot(xvals, luckiness[team], label=team + '({})'.format(num_lucky), color=get_team_color(team) , linewidth=linewidth)
    avg_list.append(luckiness[team])

# plot average
avg_arr = np.array(avg_list)
plt.plot(xvals, avg_arr.mean(axis=0), color='k', linewidth=3, label='average')

# decorate
plt.legend(loc='lower center', ncol=5)
plt.ylabel('Actual score - Projected score')
plt.xlabel('Week')
#plt.ylim([0, 225])
plt.title('Actual team score - Projected team score')
plt.xticks(xvals)


([<matplotlib.axis.XTick at 0x149065d2f08>,
 <a list of 13 Text major ticklabel objects>)

In [169]:
# plot average player rank by week for each position
pos='QB'
for team in teams:
    for week in xvals:

        # iterate players in position
        players = get_team_starters(team, week, pos=pos)
        for p in players:
            rank = get_player_performance_rank(p, week, only_starters=True)



7

['Russell Wilson']